In [11]:
"""
This code snippet imports several essential libraries for data analysis, visualization, and geospatial plotting.
Each library has a specific role, and the `%matplotlib inline` command is used to enable inline plotting in Jupyter Notebooks.
"""

# Import the NumPy library for numerical computations.
import numpy as np

# Import the Pandas library for data manipulation and analysis, particularly useful for handling tabular data.
import pandas as pd

# This magic command is specific to Jupyter Notebooks and allows Matplotlib plots to be displayed inline within the notebook.
%matplotlib inline

# Import the Matplotlib library, which is a comprehensive library for creating static, animated, and interactive visualizations in Python.
import matplotlib as mpl

# Import the Pyplot module from Matplotlib, which provides a MATLAB-like interface for creating figures and plots.
import matplotlib.pyplot as plt

# Import the Seaborn library, which is built on top of Matplotlib and provides a high-level interface for drawing attractive and informative statistical graphics.
import seaborn as sns

# Import the Folium library, which is used for creating interactive maps using Leaflet.js.
import folium

In [12]:
"""
This code snippet is designed for use with a standard Jupyter Notebook. It imports necessary libraries, fetches a CSV file from a URL,
and loads it into a Pandas DataFrame. The data is then ready for analysis within the notebook.
"""

# Import the requests library to handle HTTP requests.
import requests

# Import the io module, which provides the tools needed to handle the binary data of the response.
import io

# Import the pandas library, which is crucial for data manipulation and analysis.
import pandas as pd

# Define the URL of the CSV file to be downloaded.
URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv"

# Send a GET request to the URL to fetch the content of the CSV file.
response = requests.get(URL)

# Check if the request was successful.
if response.status_code == 200:
    # Convert the response content into a BytesIO object, which allows it to be handled as a file-like object by pandas.
    data = io.BytesIO(response.content)
    
    # Read the CSV data from the BytesIO object into a Pandas DataFrame.
    df = pd.read_csv(data)
    
    # Print a confirmation message to indicate that the data has been successfully downloaded and loaded into a DataFrame.
    print('Data downloaded and read into a dataframe!')
else:
    # Print an error message if the request was not successful.
    print(f"Failed to download data. HTTP Status code: {response.status_code}")

Data downloaded and read into a dataframe!


In [15]:
"""
This script creates a Dash application with a centered title, drop-down menus, and a division for displaying output.
The title is styled with a specific color and font size.
The drop-down menus allow users to select report types and years.
The application includes a division for displaying output and callback functions to update the UI based on user interactions.
"""

# Import necessary libraries
import dash  # Dash is a framework for building analytical web applications.
from dash import html, dcc  # Import HTML components from Dash.
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Create the Dash app instance
app = dash.Dash(__name__)

# Use the pre-loaded DataFrame `df` from the previous code snippet
# Assuming df has been defined earlier with the fetched data
df['Year'] = pd.DatetimeIndex(df['Date']).year

# Define a list of years from the DataFrame
year_list = df['Year'].unique()

# Define the layout of the Dash app
app.layout = html.Div(children=[
    # Task 2.1: Create a Dash application title
    html.H1(
        "Automobile Sales Statistics Dashboard",  # Title of the dashboard
        style={
            'textAlign': 'center',  # CSS property to center align the text
            'color': '#503D36',  # CSS property to set the text color
            'font-size': '24px'  # CSS property to set the font size
        }
    ),
    
    # Task 2.2: Add drop-down menus to the dashboard
    dcc.Dropdown(
        id='dropdown-statistics',  # Unique identifier for the dropdown
        options=[
            {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},  # Option 1: Yearly Statistics
            {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}  # Option 2: Recession Period Statistics
        ],
        placeholder='Select a report type',  # Placeholder text when no option is selected
        value='Yearly Statistics',  # Default value
        style={
            'width': '80%',  # CSS property to set the width of the dropdown to 80% of the container width
            'padding': '3px',  # CSS property to set the padding inside the dropdown
            'font-size': '20px',  # CSS property to set the font size of the dropdown text
            'text-align-last': 'center'  # CSS property to center-align the selected value in the dropdown
        }
    ),
    
    dcc.Dropdown(
        id='select-year',  # Unique identifier for the dropdown
        options=[{'label': str(i), 'value': i} for i in year_list],  # Generate options from the year_list using list comprehension
        placeholder='Select year',  # Placeholder text when no option is selected
        value=str(year_list[0]),  # Default value
        style={
            'width': '80%',  # CSS property to set the width of the dropdown to 80% of the container width
            'padding': '3px',  # CSS property to set the padding inside the dropdown
            'font-size': '20px',  # CSS property to set the font size of the dropdown text
            'text-align-last': 'center'  # CSS property to center-align the selected value in the dropdown
        }
    ),

    # Task 2.3: Add a division for output display with appropriate id and classname property
    html.Div([
        html.Div(id='output-container', className='chart-grid', style={'display': 'flex'})
    ])
])

# Task 2.4: Create callback to enable or disable the year selection dropdown
@app.callback(
    Output('select-year', 'disabled'),
    Input('dropdown-statistics', 'value')
)
def update_year_dropdown(selected_statistics):
    """
    This function enables or disables the year selection dropdown based on the user's choice of report type.
    - If 'Yearly Statistics' is selected, the year dropdown is enabled.
    - If 'Recession Period Statistics' is selected, the year dropdown is disabled.
    
    Parameters:
    selected_statistics (str): The value of the selected statistics report type.
    
    Returns:
    bool: False if 'Yearly Statistics' is selected (enables the dropdown), True otherwise (disables the dropdown).
    """
    if selected_statistics == 'Yearly Statistics':
        return False  # Enable the dropdown
    else:
        return True  # Disable the dropdown

# Task 2.4 and Task 2.5: Create callback to update the output container with graphs
@app.callback(
    Output('output-container', 'children'),
    [Input('dropdown-statistics', 'value'), Input('select-year', 'value')]
)
def update_output_container(selected_statistics, selected_year):
    """
    This function generates and returns graphs based on the selected report type and year.
    - For 'Recession Period Statistics', filters data for recession periods and creates four plots.
    - For 'Yearly Statistics', filters data for the selected year and creates four plots.
    
    Parameters:
    selected_statistics (str): The value of the selected statistics report type.
    selected_year (str or int): The selected year.
    
    Returns:
    list: A list of Dash HTML Div components containing the generated graphs.
    """
    if selected_statistics == 'Recession Period Statistics':
        # Filter the data for recession periods
        recession_data = df[df['Recession'] == 1]
        
        # Plot 1: Automobile sales fluctuate over Recession Period (year-wise) using line chart
        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(
            figure=px.line(yearly_rec, 
                           x='Year', 
                           y='Automobile_Sales', 
                           title="Automobile Sales over Recession Period")
        )
        
        # Plot 2: Average number of vehicles sold by vehicle type as a Bar chart
        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2 = dcc.Graph(
            figure=px.bar(average_sales, 
                          x='Vehicle_Type', 
                          y='Automobile_Sales', 
                          title="Average Number of Vehicles Sold by Vehicle Type during Recession")
        )
        
        # Plot 3: Pie chart for total expenditure share by vehicle type during recessions
        exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(
            figure=px.pie(exp_rec, 
                          values='Advertising_Expenditure', 
                          names='Vehicle_Type', 
                          title="Total Expenditure Share by Vehicle Type during Recession")
        )
        
        # Plot 4: Bar chart for the effect of unemployment rate on vehicle type and sales
        unemp_data = recession_data.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
        R_chart4 = dcc.Graph(
            figure=px.bar(unemp_data, 
                          x='unemployment_rate', 
                          y='Automobile_Sales', 
                          color='Vehicle_Type', 
                          labels={'unemployment_rate': 'Unemployment Rate', 'Automobile_Sales': 'Average Automobile Sales'},
                          title='Effect of Unemployment Rate on Vehicle Type and Sales')
        )
        
        # Return the generated plots as a list of HTML Div components
        return [
            html.Div(className='chart-item', children=[R_chart1, R_chart2], style={'display': 'flex'}),
            html.Div(className='chart-item', children=[R_chart3, R_chart4], style={'display': 'flex'})
        ]
    
    elif selected_statistics == 'Yearly Statistics' and selected_year:
        # Filter the data for the selected year
        yearly_data = df[df['Year'] == int(selected_year)]
        
        # Plot 1: Yearly Automobile sales using line chart for the whole period
        yas = df.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(
            figure=px.line(yas, 
                           x='Year', 
                           y='Automobile_Sales', 
                           title="Yearly Automobile Sales")
        )
        
        # Plot 2: Total Monthly Automobile sales using line chart
        mas = yearly_data.groupby('Month')['Automobile_Sales'].sum().reset_index()
        Y_chart2 = dcc.Graph(
            figure=px.line(mas, 
                           x='Month', 
                           y='Automobile_Sales', 
                           title='Total Monthly Automobile Sales')
        )
        
        # Plot 3: Bar chart for average number of vehicles sold during the given year
        avr_vdata = yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        Y_chart3 = dcc.Graph(
            figure=px.bar(avr_vdata, 
                          x='Vehicle_Type', 
                          y='Automobile_Sales', 
                          title=f'Average Vehicles Sold by Vehicle Type in {selected_year}')
        )
        
        # Plot 4: Total Advertisement Expenditure for each vehicle using pie chart
        exp_data = yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        Y_chart4 = dcc.Graph(
            figure=px.pie(exp_data, 
                          values='Advertising_Expenditure', 
                          names='Vehicle_Type', 
                          title='Total Advertisement Expenditure for Each Vehicle')
        )
        
        # Return the generated plots as a list of HTML Div components
        return [
            html.Div(className='chart-item', children=[Y_chart1, Y_chart2], style={'display': 'flex'}),
            html.Div(className='chart-item', children=[Y_chart3, Y_chart4], style={'display': 'flex'})
        ]

    else:
        # If no valid option is selected, return an empty list
        return []

# Run the Dash app server
# The '__name__' variable is a special Python variable that represents the name of the module.
# If the script is run directly, '__name__' will be '__main__', and the app will start.
if __name__ == '__main__':
    # 'app.run_server' starts the Flask server.
    # 'port' parameter specifies the port number to run the server on. Default is 8050, but we use 8066 to avoid conflicts.
    # 'debug=True' enables debug mode, which provides more detailed error messages and auto-reloads the app on code changes.
    app.run_server(port=8066, debug=True)

In [ ]:
# Import necessary libraries
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Create the Dash app instance
app = dash.Dash(__name__)

# Use the pre-loaded DataFrame `df` from the previous code snippet
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv")
df['Year'] = pd.DatetimeIndex(df['Date']).year

# Define a list of years from the DataFrame
year_list = df['Year'].unique()

# Define the layout of the Dash app
app.layout = html.Div(children=[
    html.H1(
        "Automobile Sales Statistics Dashboard",
        style={'textAlign': 'center', 'color': '#503D36', 'font-size': '24px'}
    )
])

# Run the Dash app server
if __name__ == '__main__':
    app.run_server(port=8066, debug=True)


In [ ]:
# Import necessary libraries
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Create the Dash app instance
app = dash.Dash(__name__)

# Use the pre-loaded DataFrame `df` from the previous code snippet
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv")
df['Year'] = pd.DatetimeIndex(df['Date']).year

# Define a list of years from the DataFrame
year_list = df['Year'].unique()

# Define the layout of the Dash app
app.layout = html.Div(children=[
    html.H1(
        "Automobile Sales Statistics Dashboard",
        style={'textAlign': 'center', 'color': '#503D36', 'font-size': '24px'}
    ),
    dcc.Dropdown(
        id='dropdown-statistics',
        options=[
            {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
            {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
        ],
        placeholder='Select a report type',
        value='Yearly Statistics',
        style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}
    ),
    dcc.Dropdown(
        id='select-year',
        options=[{'label': str(i), 'value': i} for i in year_list],
        placeholder='Select year',
        value=str(year_list[0]),
        style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}
    )
])

# Run the Dash app server
if __name__ == '__main__':
    app.run_server(port=8066, debug=True)


In [ ]:
# Import necessary libraries
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Create the Dash app instance
app = dash.Dash(__name__)

# Use the pre-loaded DataFrame `df` from the previous code snippet
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv")
df['Year'] = pd.DatetimeIndex(df['Date']).year

# Define a list of years from the DataFrame
year_list = df['Year'].unique()

# Define the layout of the Dash app
app.layout = html.Div(children=[
    html.H1(
        "Automobile Sales Statistics Dashboard",
        style={'textAlign': 'center', 'color': '#503D36', 'font-size': '24px'}
    ),
    dcc.Dropdown(
        id='dropdown-statistics',
        options=[
            {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
            {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
        ],
        placeholder='Select a report type',
        value='Yearly Statistics',
        style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}
    ),
    dcc.Dropdown(
        id='select-year',
        options=[{'label': str(i), 'value': i} for i in year_list],
        placeholder='Select year',
        value=str(year_list[0]),
        style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}
    ),
    html.Div([
        html.Div(id='output-container', className='chart-grid', style={'display': 'flex'})
    ])
])

# Run the Dash app server
if __name__ == '__main__':
    app.run_server(port=8066, debug=True)

In [ ]:
# Import necessary libraries
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Create the Dash app instance
app = dash.Dash(__name__)

# Use the pre-loaded DataFrame `df` from the previous code snippet
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv")
df['Year'] = pd.DatetimeIndex(df['Date']).year

# Define a list of years from the DataFrame
year_list = df['Year'].unique()

# Define the layout of the Dash app
app.layout = html.Div(children=[
    html.H1(
        "Automobile Sales Statistics Dashboard",
        style={'textAlign': 'center', 'color': '#503D36', 'font-size': '24px'}
    ),
    dcc.Dropdown(
        id='dropdown-statistics',
        options=[
            {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
            {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
        ],
        placeholder='Select a report type',
        value='Yearly Statistics',
        style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}
    ),
    dcc.Dropdown(
        id='select-year',
        options=[{'label': str(i), 'value': i} for i in year_list],
        placeholder='Select year',
        value=str(year_list[0]),
        style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}
    ),
    html.Div([
        html.Div(id='output-container', className='chart-grid', style={'display': 'flex'})
    ])
])

@app.callback(
    Output('select-year', 'disabled'),
    Input('dropdown-statistics', 'value')
)
def update_year_dropdown(selected_statistics):
    if selected_statistics == 'Yearly Statistics':
        return False
    else:
        return True

# Run the Dash app server
if __name__ == '__main__':
    app.run_server(port=8066, debug=True)

In [ ]:
# Import necessary libraries
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Create the Dash app instance
app = dash.Dash(__name__)

# Use the pre-loaded DataFrame `df` from the previous code snippet
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv")
df['Year'] = pd.DatetimeIndex(df['Date']).year

# Define a list of years from the DataFrame
year_list = df['Year'].unique()

# Define the layout of the Dash app
app.layout = html.Div(children=[
    html.H1(
        "Automobile Sales Statistics Dashboard",
        style={'textAlign': 'center', 'color': '#503D36', 'font-size': '24px'}
    ),
    dcc.Dropdown(
        id='dropdown-statistics',
        options=[
            {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
            {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
        ],
        placeholder='Select a report type',
        value='Yearly Statistics',
        style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}
    ),
    dcc.Dropdown(
        id='select-year',
        options=[{'label': str(i), 'value': i} for i in year_list],
        placeholder='Select year',
        value=str(year_list[0]),
        style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}
    ),
    html.Div([
        html.Div(id='output-container', className='chart-grid', style={'display': 'flex'})
    ])
])

@app.callback(
    Output('select-year', 'disabled'),
    Input('dropdown-statistics', 'value')
)
def update_year_dropdown(selected_statistics):
    if selected_statistics == 'Yearly Statistics':
        return False
    else:
        return True

@app.callback(
    Output('output-container', 'children'),
    [Input('dropdown-statistics', 'value'), Input('select-year', 'value')]
)
def update_output_container(selected_statistics, selected_year):
    if selected_statistics == 'Recession Period Statistics':
        recession_data = df[df['Recession'] == 1]
        
        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(
            figure=px.line(yearly_rec, 
                           x='Year', 
                           y='Automobile_Sales', 
                           title="Automobile Sales over Recession Period")
        )
        
        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2 = dcc.Graph(
            figure=px.bar(average_sales, 
                          x='Vehicle_Type', 
                          y='Automobile_Sales', 
                          title="Average Number of Vehicles Sold by Vehicle Type during Recession")
        )
        
        exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(
            figure=px.pie(exp_rec, 
                          values='Advertising_Expenditure', 
                          names='Vehicle_Type', 
                          title="Total Expenditure Share by Vehicle Type during Recession")
        )
        
        unemp_data = recession_data.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
        R_chart4 = dcc.Graph(
            figure=px.bar(unemp_data, 
                          x='unemployment_rate', 
                          y='Automobile_Sales', 
                          color='Vehicle_Type', 
                          labels={'unemployment_rate': 'Unemployment Rate', 'Automobile_Sales': 'Average Automobile Sales'},
                          title='Effect of Unemployment Rate on Vehicle Type and Sales')
        )
        
        return [
            html.Div(className='chart-item', children=[R_chart1, R_chart2], style={'display': 'flex'}),
            html.Div(className='chart-item', children=[R_chart3, R_chart4], style={'display': 'flex'})
        ]
    
    return []

# Run the Dash app server
if __name__ == '__main__':
    app.run_server(port=8066, debug=True)

In [ ]:
# Import necessary libraries
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Create the Dash app instance
app = dash.Dash(__name__)

# Use the pre-loaded DataFrame `df` from the previous code snippet
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv")
df['Year'] = pd.DatetimeIndex(df['Date']).year

# Define a list of years from the DataFrame
year_list = df['Year'].unique()

# Define the layout of the Dash app
app.layout = html.Div(children=[
    html.H1(
        "Automobile Sales Statistics Dashboard",
        style={'textAlign': 'center', 'color': '#503D36', 'font-size': '24px'}
    ),
    dcc.Dropdown(
        id='dropdown-statistics',
        options=[
            {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
            {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
        ],
        placeholder='Select a report type',
        value='Yearly Statistics',
        style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}
    ),
    dcc.Dropdown(
        id='select-year',
        options=[{'label': str(i), 'value': i} for i in year_list],
        placeholder='Select year',
        value=str(year_list[0]),
        style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}
    ),
    html.Div([
        html.Div(id='output-container', className='chart-grid', style={'display': 'flex'})
    ])
])

@app.callback(
    Output('select-year', 'disabled'),
    Input('dropdown-statistics', 'value')
)
def update_year_dropdown(selected_statistics):
    if selected_statistics == 'Yearly Statistics':
        return False
    else:
        return True

@app.callback(
    Output('output-container', 'children'),
    [Input('dropdown-statistics', 'value'), Input('select-year', 'value')]
)
def update_output_container(selected_statistics, selected_year):
    if selected_statistics == 'Recession Period Statistics':
        recession_data = df[df['Recession'] == 1]
        
        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(
            figure=px.line(yearly_rec, 
                           x='Year', 
                           y='Automobile_Sales', 
                           title="Automobile Sales over Recession Period")
        )
        
        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2 = dcc.Graph(
            figure=px.bar(average_sales, 
                          x='Vehicle_Type', 
                          y='Automobile_Sales', 
                          title="Average Number of Vehicles Sold by Vehicle Type during Recession")
        )
        
        exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(
            figure=px.pie(exp_rec, 
                          values='Advertising_Expenditure', 
                          names='Vehicle_Type', 
                          title="Total Expenditure Share by Vehicle Type during Recession")
        )
        
        unemp_data = recession_data.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
        R_chart4 = dcc.Graph(
            figure=px.bar(unemp_data, 
                          x='unemployment_rate', 
                          y='Automobile_Sales', 
                          color='Vehicle_Type', 
                          labels={'unemployment_rate': 'Unemployment Rate', 'Automobile_Sales': 'Average Automobile Sales'},
                          title='Effect of Unemployment Rate on Vehicle Type and Sales')
        )
        
        return [
            html.Div(className='chart-item', children=[R_chart1, R_chart2], style={'display': 'flex'}),
            html.Div(className='chart-item', children=[R_chart3, R_chart4], style={'display': 'flex'})
        ]
    
    elif selected_statistics == 'Yearly Statistics' and selected_year:
        yearly_data = df[df['Year'] == int(selected_year)]
        
        yas = df.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(
            figure=px.line(yas, 
                           x='Year', 
                           y='Automobile_Sales', 
                           title="Yearly Automobile Sales")
        )
        
        mas = yearly_data.groupby('Month')['Automobile_Sales'].sum().reset_index()
        Y_chart2 = dcc.Graph(
            figure=px.line(mas, 
                           x='Month', 
                           y='Automobile_Sales', 
                           title='Total Monthly Automobile Sales')
        )
        
        avr_vdata = yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        Y_chart3 = dcc.Graph(
            figure=px.bar(avr_vdata, 
                          x='Vehicle_Type', 
                          y='Automobile_Sales', 
                          title=f'Average Vehicles Sold by Vehicle Type in {selected_year}')
        )
        
        exp_data = yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        Y_chart4 = dcc.Graph(
            figure=px.pie(exp_data, 
                          values='Advertising_Expenditure', 
                          names='Vehicle_Type', 
                          title='Total Advertisement Expenditure for Each Vehicle')
        )
        
        return [
            html.Div(className='chart-item', children=[Y_chart1, Y_chart2], style={'display': 'flex'}),
            html.Div(className='chart-item', children=[Y_chart3, Y_chart4], style={'display': 'flex'})
        ]

    return []

# Run the Dash app server
if __name__ == '__main__':
    app.run_server(port=8066, debug=True)
